<a href="https://colab.research.google.com/github/ilana27/Brown-Deep-Learning-hw1-ilana27/blob/main/HW1_Linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS1470/2470 HW1: Single-Layered Neural Networks

In this homework assignment, you will build a simple linear model using differential modules.

---

## Starting Our Modular API

### **The goal of this assignment is as follows:** 
- Organize our understanding of deep learning into a modular framework. 
- Get familiarized with a simple modular API which reflects (but is a simplification of) some of PyTorch's systems. 
- Implement some nice modular components and be able to construct a functional single-file neural network from it. 

### Improving The Old Way via Chain Rule

Recall that in the machine learning lab, we got the chance to make some simple but effective regression models! Given some realizations $X \sim \mathcal{X}$ and $Y = \mathbb{E}[Y|X] + \xi$, we were able to train up a model $h_{\theta} \in \mathcal{H}$ which was similar to $\mathbb{E}[Y|X]$ and thereby minimized an empirical loss $\mathcal{L}$ of our choice.

> As a reminder, $\mathcal{X}$ is the space of all candidate functions.

In these cases, we assumed that $h_{\theta}$ had a relatively simple and non-flexible architecture, which meant that we could manually specify the structure and simply derive and code up our gradient formula once. Furthermore, since the optimization process was concave, we could even skip the gradient computation and directly derive a loss-minimizing parameter selection. 

Of course, there are hard limits to what this kind of architecture can provide us; sometimes the relationships that the model needs to capture are relatively complex and might not be resolvable in such a fashion. And that's why this course exists!

#### **Problem:** 
> This is extremely time-consuming and rigid! What happens if we switched out an activation function? A loss function? We'd have to re-specify the gradient every time!

#### **Solution:** 
> Let's take advantage of the chain rule! 

**Naive Solution:** If we really wanted to, we could approach this problem in the same way as before, and just code up the gradient functions manually. Similarly to before, this would allow us to propagate gradients through, say, a specified loss function, an activation function, and a dense layer. We could also do it for 2 dense layers; just use the old gradient function for the weights in layer 2, compute the new gradient for the weights in layer 1, and so on. 

Recall that per the chain rule, if there exists a set of differentiable functions $c(b)$ and $b(a)$, then 

$$\frac{\partial a}{\partial c} = \frac{\partial a}{\partial b} \frac{\partial b}{\partial c}$$

Going back to our regression model, let's assume that we have a layered process: 

$$x \to h_\theta(x) \to \mathcal{L}(h_\theta)$$

This implies that we can compute the partial of the trainable parameters $\theta$ through a loss evaluation $\mathcal{L}$ and a dense layer $h_\theta(x)$ by the following relationship:

$$\frac{\partial \mathcal{L}}{\partial \theta} = \frac{\partial \mathcal{h_\theta}}{\partial \theta}\frac{\partial \mathcal{L}}{\partial h_\theta}$$

With a similar logic, you can also make the assertion that you can also get the partial with respect to the input $x$:

$$\frac{\partial \mathcal{L}}{\partial x} = \frac{\partial h_\theta}{\partial x}\frac{\partial \mathcal{L}}{\partial h_\theta}$$

So... by the same token, is there anything stopping us from going further? Let's say that we decided to have another hypothesis function such that $x = h'_{\theta'}(x')$ for some other hypothesis function and inputs? The new structure would then be: 

$$x' \to \big[ x = h'_{\theta'}(x') \big] \to h_\theta(x) \to \mathcal{L}(h_\theta)$$

Without the chain rule, coding in the facilities to optimize $\theta'$ might have been tricky, but with the chain rule we know that:

$$\frac{\partial \mathcal{L}}{\partial \theta'} 
= \frac{\partial x}{\partial \theta'}\frac{\partial h}{\partial x}\frac{\partial \mathcal{L}}{\partial h} 
= \frac{\partial x}{\partial \theta'}\frac{\partial \mathcal{L}}{\partial x}$$

Notice how this process is both predictable and scales very well! Say that we wanted to add some activation functions to restrict the range of the hypothesis functions. This trivially inserts into the chain and everything still works and will look something like this: 

$$ \frac{\partial \mathcal{L}}{\partial x} = \frac{\partial h}{\partial x}\frac{\partial a}{\partial h}\frac{\partial \mathcal{L}}{\partial a} 
\ \text{ and } \ \frac{\partial \mathcal{L}}{\partial \theta} = \frac{\partial h}{\partial \theta}\frac{\partial a}{\partial h}\frac{\partial \mathcal{L}}{\partial a} 
$$

And with that, we start to approach the reason why this is such a powerful formulation: The cumulative nature of the process. Specifically, consider the process that needs to happen in order to compute this for the extended 2-layer example: 

$$
\begin{align}
\frac{\partial a}{\partial h}\frac{\partial \mathcal{L}}{\partial a} 
= \frac{\partial \mathcal{L}}{\partial h} 
&\to \cdots = \frac{\partial \mathcal{L}}{\partial x} 
&\to \cdots = \frac{\partial \mathcal{L}}{\partial a'}
&\to \cdots = \frac{\partial \mathcal{L}}{\partial h'} 
&\to \cdots = \frac{\partial \mathcal{L}}{\partial x'} 
\\
&\searrow \cdots = \frac{\partial \mathcal{L}}{\partial \theta} 
&&&\searrow \cdots = \frac{\partial \mathcal{L}}{\partial \theta'} 
% \\
% &\searrow \cdots = \frac{\partial \mathcal{L}}{\partial \theta} 
% \to \cdots = \frac{\partial \mathcal{L}}{\partial a'} 
% \to \cdots = \frac{\partial \mathcal{L}}{\partial h'} 
% &
% \to \cdots = \frac{\partial \mathcal{L}}{\partial \theta'} 
% \\ 
% &&
% \to \cdots = \frac{\partial \mathcal{L}}{\partial x'} 
\end{align}
$$

... and this is the process known as **back-propagation** *(and a special case of **auto-differentiation**)*!

-----

## Loading In Our Data

The first thing we have to do is load in our data to use our model with. We'll be working with the [diabetes dataset from the sklearn package](https://scikit-learn.org/stable/datasets/toy_dataset.html#diabetes-dataset). We've already provided the code to load in the input data and the ground truth labels (stored as `X` and `Y` respectively below).

**[TODO]:** Split the samples into training and testing sets. We'll train with the train set and reserve the testing set to evaluate the model's performance on samples it hasn't seen.

The diabetes dataset has 442 samples. Each sample's input data has 10 data points for some key metrics, like age and cholesterol levels. The "label" is a number representing disease progression one year after baseline. Thus, `X` has shape `(442, 10)`, while `Y` has shape `(442,)`.

**[TODO]:** Reshape the `Y` subsets to have shape `(num_samples, 1)`.

In [94]:
import numpy as np
from sklearn.datasets import load_diabetes


diabetes = load_diabetes()
X, Y = diabetes.data, diabetes.target

## TODO: Split the data into a 80%-20% training-testing split
## TODO: Reshape the Y subsets to have shape (num_samples, 1)
# 422 rows, 10 columns. need 80% of those rows
train_size = int(0.2*len(X))
X0 = X[train_size : int(len(X))]
X1 = X[0: train_size]
Y0 = Y[train_size : int(len(X))].reshape(-1, 1)
Y1 = Y[0: train_size].reshape(-1, 1)


print(f"""
> Input shape: {X0.shape} for training, {X1.shape} for testing
> Label shape: {Y0.shape} for training, {Y1.shape} for testing
""".strip())

> Input shape: (354, 10) for training, (88, 10) for testing
> Label shape: (354, 1) for training, (88, 1) for testing


## **Exploring a possible modular implementation: PyTorch**

Next, we'll want to build up a Regression model interface, from which we can implement specific regression model classes, like the LinearRegression class we'll work with later. 

We subclass the `nn.Module class`, which represents any module (like a layer or model) of a deep learning system.

In [95]:
import torch
import torch.nn as nn

class Regression(nn.Module):

    """
    Initialize all the inherent "things" inside of a model!
    This includes things like the layers, activation/loss functions, and optimzer. 
    """
    def __init__(self, input_dims, output_dims):
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")        
        super().__init__()
        self.dense = nn.Linear(input_dims, output_dims).to(self.device)   
        self.activation = None  ## To be specified in subclasses 
        self.loss = None        ## To be specified in subclasses 
        self.set_learning_rate()

    """
    Sets up the optmizer
    """
    def set_learning_rate(self, learning_rate=0.001):
        self.optimizer = torch.optim.SGD(self.parameters(), lr=learning_rate) ## Simple stochastic gradient descent (SGD) optimizer

    """
    Forward pass of the model
    Given an input x, how does the model process the input to get its output?
    """
    def forward(self, x):
        x = self.dense(x)
        x = self.activation(x)
        return x


## Adding a PyTorch Training/Evaluation Routine

Now that we have the basis for a Regression model, we need to describe how to train (`fit`) and evaluate (`evaluate`) our model, given data. 
Every epoch, we want to fit our model to the training data, and then evaluate our model on the testing data.

In [96]:
class TrainTest:

    no_grad = torch.no_grad

    def fit(self, data):
        ## Training loop
        self.train()        ## Set model into training mode
        ## Iterate over the data batches
        for batch, (inputs, target) in enumerate(data):
            ## In real pytorch, you'd need to set the device
            inputs = inputs.to(self.device)
            target = target.to(self.device)
            ## Erase the gradient history
            self.optimizer.zero_grad()
            ## Do a forward pass on the model
            output = self(inputs)
            ## Compute the loss
            loss = self.loss(output, target)
            ## Run backwards pass from the loss through the previous layers
            ## This will accumulate gradients for the parameters that need to be optimized
            loss.backward()
            ## Perform a single optimization step
            self.optimizer.step()
        return {'loss' : loss}

    def evaluate(self, data):
        ## Set model into "evaluate" mode so that the parameters don't get updated
        self.eval()
        total_loss = 0
        ## Cut off the tensor training scope to make sure weights aren't updated
        ## For now, it's torch.no_grad; later, you'll use Tensor.no_grad
        with TrainTest.no_grad():
            for inputs, target in data:
                ## In real pytorch, you'd need to set the device
                inputs = inputs.to(self.device)
                target = target.to(self.device)
                output = self(inputs)
                total_loss += self.loss(output, target).item()  # sum up batch loss

        total_loss /= len(data)
        return {'test_loss' : total_loss}
        
    def train_test(self, train_data, test_data, epochs=1):
        ## Does both training and validation on a per-epoch basis
        all_stats = []
        for epoch in range(epochs):
            train_stats = self.fit(train_data)
            test_stats = self.evaluate(test_data)
            all_stats += [{**train_stats, **test_stats}]
            print(f'[Epoch {epoch+1}/{epochs}]', all_stats[-1])
        return all_stats

## Making Linear Regression with Train/Test Capabilities

Next, we implement the `LinearRegression` class, which subclasses both `Regression` and `TrainTest` to inherit useful methods. 

Consider a dense layer represented by $Y = XA + B$ where: 
- $X$ has shape `(n,input_dims)`
- $A$ has shape `(input_dims,output_dims)`
- $B$ has shape `(n,)`
- $Y$ has shape `(n, output_dims)`. 

Let each row of $X, B, Y$ (or the 0$^\text{th}$ dimension) represent a different sample. Disregarding bias, note how any given row in $Y$ is a set of linear combinations of the same row in $X$. In other words, a Dense Layer is high-dimensional Linear Regression. 

Once you've built up your model, try training and testing it in the code block below!

In [97]:
class LinearRegression(Regression, TrainTest):
    def __init__(self, input_dims, output_dims):
        super().__init__(input_dims, output_dims)
        self.activation = nn.Identity()
        self.loss = nn.MSELoss()

In [98]:
torch_model = LinearRegression(X0.shape[-1], 1)
torch_model.set_learning_rate(0.3)
torch_model.train_test(
    [[torch.Tensor(X0), torch.Tensor(Y0)]], 
    [[torch.Tensor(X1), torch.Tensor(Y1)]],
    epochs=200
);

[Epoch 1/200] {'loss': tensor(30486.2070, grad_fn=<MseLossBackward0>), 'test_loss': 6578.00732421875}
[Epoch 2/200] {'loss': tensor(9980.0781, grad_fn=<MseLossBackward0>), 'test_loss': 4815.94091796875}
[Epoch 3/200] {'loss': tensor(6678.2480, grad_fn=<MseLossBackward0>), 'test_loss': 4881.97802734375}
[Epoch 4/200] {'loss': tensor(6128.7046, grad_fn=<MseLossBackward0>), 'test_loss': 5021.32666015625}
[Epoch 5/200] {'loss': tensor(6019.6396, grad_fn=<MseLossBackward0>), 'test_loss': 5085.34423828125}
[Epoch 6/200] {'loss': tensor(5981.2471, grad_fn=<MseLossBackward0>), 'test_loss': 5102.7861328125}
[Epoch 7/200] {'loss': tensor(5954.3652, grad_fn=<MseLossBackward0>), 'test_loss': 5099.14892578125}
[Epoch 8/200] {'loss': tensor(5929.5298, grad_fn=<MseLossBackward0>), 'test_loss': 5086.82080078125}
[Epoch 9/200] {'loss': tensor(5905.2251, grad_fn=<MseLossBackward0>), 'test_loss': 5071.09033203125}
[Epoch 10/200] {'loss': tensor(5881.2056, grad_fn=<MseLossBackward0>), 'test_loss': 5054.11

## Shapes That Might Be Useful...

Throughout the duration of this course, you might find it really helpful to check the shapes of each of your different tensors and outputs just to verify that everything is working as intended. Check the block below to see an example!

In [99]:
y_true = torch.Tensor(Y0)
y_pred = torch_model(torch.Tensor(X0))
loss = torch_model.loss(y_true, y_pred)

print(f"""
> Prediction Shape: {y_pred.shape}
> Weights    Shape: {list(torch_model.parameters())[0].shape}
> Bias       Shape: {list(torch_model.parameters())[1].shape}
> Loss       Shape: {loss.shape}
""".strip())

> Prediction Shape: torch.Size([354, 1])
> Weights    Shape: torch.Size([1, 10])
> Bias       Shape: torch.Size([1])
> Loss       Shape: torch.Size([])


Next, let's start building up all the different parts of the basic PyTorch tools that we used in order to see what's under the hood.

## PyTorch Complexity Assumptions

### Tensors
- Tensors are responsible for maintaining their own gradients
- Tensors hold on to `backward` functions to which they can pass a gradient into. These backwards functions are provided by the layers associated with those tensors. 
    - If a tensor is a terminal node, it will pass in an upstream gradient of `None`.
    - If a tensor is a non-terminal node, it will pass the accumulated upstream gradient. 
    - `backward` functions as a linked list algorithm and crawls back the chain, computing the gradient for every tensor that it hits (as long as they require a gradient).
- Since the tensors hold their own gradients, the optimizer can merely take the tensors' values, take their gradients, and then just optimize them.
- However, because tensors are always keeping track of their gradients when `requires_grad` is set to `True`, we also want to add a way to stop tracking gradients.
    - For instance, while evaluating the performance of our model, we want to make sure that the model doesn't learn anything from this evaluation phase.
    - We can use the `no_grad` subclass to automatically handle the flipping of this `requires_grad` value. 
        - Every time we enter a `with Tensor.no_grad():` block, the code within `no_grad`'s `__enter__()` method will execute. 
        - Once we exit the same `with Tensor.no_grad():` block, `no_grad`'s `__exit()__` method will run.

In [100]:
class Tensor(np.ndarray):

    '''
    Subclassing numpy arrays is a bit weird:
    https://numpy.org/doc/stable/user/basics.subclassing.html

    Just assume that the attributes referred to in __new__/__array_finalize__ 
    will be accessible in a Tensor when a new Tensor object is created.  
    '''

    requires_grad = True  ## Class variable; accessible by Tensor.requires_grad

    def __new__(cls, input_array):
        obj = np.asarray(input_array).view(cls)
        obj.backward = lambda x: None   ## Backward starts as None, gets assigned later
        obj.grad = None                 ## Gradient starts as None, gets computed later
        obj.requires_grad = True        ## By default, we'll want to compute gradient for new tensors
        obj.to = lambda x: obj          ## We don't handle special device support (i.e. cpu vs gpu/cuda)
        return obj

    def __array_finalize__(self, obj):
        if obj is None: return
        self.backward       = getattr(obj, 'backward',      lambda x: None)
        self.to             = getattr(obj, 'to',            lambda x: obj)
        self.grad           = getattr(obj, 'grad',          None)
        self.requires_grad  = getattr(obj, 'requires_grad', None)

    class no_grad():

        '''
        Synergizes with Tensor: By entering the tensor with no_grad scope, 
        the Tensor.requires_grad singleton will swap to False. 
        '''
        
        def __enter__(self):
            # When tape scope is entered, stop asking tensors to record gradients
            Tensor.requires_grad = False
            return self

        def __exit__(self, exc_type, exc_val, exc_tb):
            # When tape scope is exited, let Diffable start recording to self.operation
            Tensor.requires_grad = True

### Diffable

Let's specify a "Diffable" object which will represent a module that can be differentiated. This class will make the following assumptions: 
- Gradients will need to flow through the input pathways in order to compute earlier gradients. 
    - Therefore, inputs will need an appropriate "backward"
- Parameters will need to recieve gradients.
- More specifically, if a `Diffable` object performs an operation on some input, then we know that the gradient from the output of the Diffable w.r.t. the inputs is the gradient of the `Diffable`'s operations w.r.t. its inputs.
  - Thus, a `Diffable`'s `input_gradients()` function should return a tuple with each of the partial derivatives of the operations performed in the forward pass.

In [101]:
from abc import ABC, abstractmethod  # # For abstract method support

class Diffable(ABC):
    """
        We use these to represent differentiable layers which we can compute gradients for.
    """

    def to(self, device):
        return self         # Just there to ignore device setting calls
    
    def __call__(self, *args, **kwargs):
        
        ## The call method keeps track of method inputs and outputs
        self.argnames   = self.forward.__code__.co_varnames[1:]
        named_args      = {self.argnames[i] : args[i] for i in range(len(args))}
        self.input_dict = {**named_args, **kwargs}
        self.inputs     = [self.input_dict[arg] for arg in self.argnames if arg in self.input_dict.keys()]
        self.outputs    = self.forward(*args, **kwargs)

        ## Make sure outputs are tensors and tie back to this layer
        list_outs = isinstance(self.outputs, list) or isinstance(self.outputs, tuple)
        if not list_outs:
            self.outputs = [self.outputs]
        self.outputs = [Tensor(out) for out in self.outputs]
        for out in self.outputs: 
            out.backward = self.backward

        # print(self.__class__.__name__.ljust(24), [v.shape for v in self.inputs], '->', [v.shape for v in self.outputs])
            
        ## And then finally, it returns the output, thereby wrapping the forward
        return self.outputs if list_outs else self.outputs[0]

    def parameters(self):
        """Returns a list of parameters"""
        return ()

    @abstractmethod
    def forward(self, x):
        """Pass inputs through function. Can store inputs and outputs as instance variables"""
        pass

    @abstractmethod
    def input_gradients(self):
        """Returns local gradient of layer output w.r.t. input"""
        pass

    def weight_gradients(self):
        """Returns local gradient of layer output w.r.t. weights"""
        return []
    
    @abstractmethod
    def backward(self, grad=np.array([[1]])):
        """
        Propagate upstream gradient backwards by composing with local gradient
        
        SCAFFOLD: 

        Differentiate with respect to layer parameters:
            For every param-gradient pair
            - If all Tensors or this tensor do not require gradients, then skip
            - Otherwise, compose upstream and local gradient
        
        Differentiate with respect to layer input:
            For every input-gradient pair
            - If all Tensors or this tensor do not require gradients, then skip
            - Otherwise, compose upstream and local gradient

        Usefulseful print boilerplate...: 
            # print(f'Diffing w.r.t. "{k}": local = {g.shape} and upstream = {grad.shape}')
        """
        pass

### Loss
**[TODO]:** Implement the forward pass in `forward()`.
- The forward pass should just give the mean squared error between `y_pred` and `y_true`.

**[TODO]:** Implement the backward pass in `backward()`.
- This should take advantage of the layer inputs as well as the gradients computed with respect to them.
- Feel free to only work with the input gradients, since this loss layer does not have any parameters.

**[TODO]:** Calculate and return `input_gradients()`:
- You want to calculate the gradients which flow to the inputs: `y_pred` and `y_true`
- Note that we don't want to "train" `y_true`, so you can just return 0 for the grads for `y_true`
- Return the partial derivative of mean squared error w.r.t. `y_pred`, and 0.

Note that we don't need to implement `weight_gradients()` here because MSELoss doesn't have weights!

In [102]:
class MSELoss(Diffable):

    """
        Calculates mean squared error loss and gradient w.r.t. inputs.
        Subclasses Diffable.
    """
    def forward(self, y_pred, y_true):
        """Mean squared error forward pass!"""
        # TODO: Compute and return the MSE given predicted and actual labels
        differences = y_pred - y_true
        return np.mean(np.square(differences))

    def input_gradients(self):
        """Mean squared error backpropagation!"""
        """Returns local gradient of layer output w.r.t. input
        """
        # TODO: Compute and return the gradients
        #obtain dL/dy_pred and dL/dy_true, dy_true = 0
        local_gradient = 2/len(self.inputs[0]) * (self.inputs[0] - self.inputs[1])
        return local_gradient, np.array([0])

    # grad is the upstream gradient
    def backward(self, grad=np.array([[1]])):
      for x, g in zip(self.inputs, self.input_gradients()):
        if (Tensor.requires_grad): 
          upstream = g @ grad
          if getattr(x, 'requires_grad', False):
            x.grad = upstream
          x.backward(upstream)        
     

And here are some sanity checks you can run to make sure that your code is working as intended. In the first check, the outputs should match. In the second, they should be within the specified range.

In [103]:
class con: 
    ## Control set using default PyTorch
    ytrue = torch.Tensor(Y0)
    ypred = torch_model(torch.Tensor(X0))
    loss_fn = nn.MSELoss()

class exp: 
    ## Experimental set using your own implementation
    ytrue = Tensor(Y0)
    ypred = Tensor(con.ypred.detach().numpy())
    loss_fn = MSELoss()

def ypred_to_loss(ns):
    ## Compute loss using the control and experimental namespaces
    ns.loss = ns.loss_fn(ns.ypred, ns.ytrue)
    return ns.loss

## Sanity Check 1: Make sure that the forward pass is the same (i.e. your implementation matches the control)
print(ypred_to_loss(con))
print(ypred_to_loss(exp))

tensor(3764.0361, grad_fn=<MseLossBackward0>)
3764.0360239159663


In [104]:
## Sanity Check 2: Make sure that the backwards pass is the same

con.ypred = con.ypred.detach()
con.ypred.requires_grad = True
# print("Before running backwards:\n", con.ypred.grad)
ypred_to_loss(con)
con.loss.backward()
# print("After running backwards:\n", con.ypred.grad)

exp.ypred.grad = None
# print("Before running backwards:\n", np.round(exp.ypred.grad, 4))
ypred_to_loss(exp)
exp.loss.backward()
# print("After running backwards:\n", np.round(exp.ypred.grad, 4))

max_diff = np.max(exp.ypred.grad - con.ypred.grad.detach().numpy())
print(f"Maximum difference {max_diff} should be less than 0.00001")

Maximum difference 2.96339477401375e-08 should be less than 0.00001


### Linear Layer
Next, a linear layer!

**[TODO]:** Implement the `forward()` pass of a linear layer. 

**[TODO]:** Calculate (manually) the weight gradients.
- Manually differentiate the Dense layer with respect to weights and biases.
- Return weight gradient, then bias gradient in that order
- HINT: How is differentiating with matrix variables similar to and different from normal differentiation?

**[TODO]:** Initalize weights and biases in `_initialize_weight()`.
- In a linear layer, we have 2 parameters: weights and biases. 
- Return two NumPy arrays of the correct shapes according according to the function's arguments. 
- Return weights and biases, in that order.

**[TODO]:** Implement the backward function.
- Feel free to only work with the weight gradients, since we do not yet need to support multilayered networks.


In [105]:
import torch
import torch.nn as nn

class Linear(Diffable):

    """
        Standard linear/dense layer.
        Subclasses Diffable.
    """

    def __init__(self, in_features, out_features, device=None, dtype=None):
        self.w, self.b = self.__class__._initialize_weight(in_features, out_features)
    
    def parameters(self):
        return self.w, self.b

    def forward(self, inputs):
        """Forward pass for a dense layer! Refer to lecture slides for how this is computed."""
        # TODO: implement the forward pass and return the outputs
        l = inputs @ self.w.T + self.b
        return l

    def weight_gradients(self):
        """Calculating the gradients of the weights and biases!"""
        # TODO: Implement calculation of gradients
        wgrads = self.input_dict["inputs"] * np.ones_like(self.w)
        bgrads = np.ones_like(self.b)
        return (wgrads, bgrads)

    def input_gradients(self):
        """Calculate the gradients of the inputs! (Not necessary for HW1)"""
        return (self.w,)

    @staticmethod
    def _initialize_weight(input_size, output_size):
        """
        Initializes the values of the weights and biases. You can assume that 
        bias is a zero-vector and weight is normally-distributed.
        """
        ## TODO: Implement default assumption: zero-init for bias, normal distribution for weights
        ## Must return tensors for tracking purposes.
        w = Tensor(np.random.normal(loc=0.0, scale=1.0, size=(output_size, input_size)))
        b = Tensor(np.zeros((output_size, 1)))
        print(b.shape)
        return w, b

    def backward(self, grad=np.array([[1]])):
        ## For every weight/bias and weight/bias gradient
            ## Compose the upstream gradient with this weight's/bias's gradient
            ## Set the gradient of the tensor to the composed gradient if necessary
            ## Backpropagate the composed gradient through the structure
        for (_, x), g in zip(enumerate(self.parameters()), self.weight_gradients()):
          if (Tensor.requires_grad): 
            upstream = np.sum(g * grad, axis = 0, keepdims = True)
            if getattr(x, 'requires_grad', False):
              x.grad = upstream
            x.backward(upstream) 

class con:
    ## Control set using regular pytorch
    X0 = torch.Tensor(X0)
    Y0 = torch.Tensor(Y0)
    X0.requires_grad = True
    Y0.requires_grad = True
    dense = nn.Linear(10, 1)
    loss_fn = nn.MSELoss()

class exp:
    ## Experimental set using your own implementation
    X0 = Tensor(X0)
    Y0 = Tensor(Y0)
    dense = Linear(10, 1)
    dense.w, dense.b = [Tensor(p.detach().numpy()) for p in con.dense.parameters()]
    loss_fn = MSELoss()

def x_to_loss(ns):
    ns.ypred = ns.dense(ns.X0)
    ns.loss  = ns.loss_fn(ns.ypred, ns.Y0)
    return ns.loss

x_to_loss(con)
x_to_loss(exp)

## Sanity Check 1: Make sure that the forward pass is the same
#print(con.ypred)
#print(exp.ypred)

print(f"Maximum difference {np.max(con.ypred.detach().numpy() - exp.ypred)} should be less than 0.00001\n")

print(f"Losses: Control {con.loss} vs Experimental {exp.loss}")

print('\nControl Params:',      *list(con.dense.parameters()), sep='\n')
print('\nExperimental Params:', *list(exp.dense.parameters()), sep='\n')

(1, 1)
Maximum difference 1.5691667965000988e-08 should be less than 0.00001

Losses: Control 30507.7421875 vs Experimental 30507.744013720378

Control Params:
Parameter containing:
tensor([[-0.1833, -0.0053, -0.0726,  0.2523, -0.1924, -0.2271, -0.0657, -0.2228,
         -0.2387, -0.2463]], requires_grad=True)
Parameter containing:
tensor([0.0648], requires_grad=True)

Experimental Params:
[[-0.18333443 -0.00532878 -0.07256863  0.25234893 -0.19242339 -0.22714584
  -0.06568353 -0.22281875 -0.23868553 -0.24626085]]
[0.06481283]


In [106]:
## Sanity Check 2: Make sure that the backwards pass is the same

con.X0 = con.X0.detach()
con.Y0 = con.Y0.detach()
for p in con.dense.parameters():
    if p.grad is None: continue
    p.grad.detach_()
    p.grad = None

x_to_loss(con).backward()
print("After running backwards on weights:")  
print([p.grad for p in con.dense.parameters()])

for p in exp.dense.parameters(): p.grad = None
x_to_loss(exp).backward()

print("\n" + "*" * 100 + "\n")
print("After running backwards on weights:")  
print([p.grad for p in exp.dense.parameters()])

After running backwards on weights:
[tensor([[-2.0612, -0.5118, -5.3810, -4.1662, -2.6959, -2.5144,  3.7024, -4.4249,
         -5.0424, -4.1924]]), tensor([-312.3180])]

****************************************************************************************************

After running backwards on weights:
[Tensor([[-2.06122185, -0.5118281 , -5.38104007, -4.16618263, -2.69592753,
         -2.51440903,  3.70239077, -4.42486337, -5.04241456, -4.19237536]]), Tensor([[-312.31796102]])]


## Optimizing With The Gradients

To use the gradients we calculated previously, we need an optimizer. The optimizer allows us to update our weights and bias. A simple approach could be to simply subtract the gradient from the weights and bias. In doing so, we follow the gradient in its opposite direction, minimizing loss. This is what is called gradient descent. 

However, simply subtracting the gradients from the weights could result in the weights changing wildly between each sample, making training longer. To prevent this, we use a learning rate. The learning rate is a hyperparameter that specifies how much a single step updates weights. A smaller learning rate means that the gradients have less of an impact on the weights, and vice versa.

Of course, this is just one (simple) approach. In a later lab, you'll learn about other optimizers, such as Adam and RMSProp.

**[TODO]:** Implement stochastic gradient descent for each parameter using the learning rate

In [107]:
class SGD: 
    """
        Performs stochastic gradient descent with the specified learning rate.
    """
    def __init__(self, params, lr, *args, **kwargs):
        self.params = params
        self.lr = lr
    
    def zero_grad(self):
        """
            Reset the gradients. to what??? go through all the parameters and set gradient to none
        """
        for p in self.params:
          p.grad = None
            
    def step(self):
        """
            Update paramaters by subtracting the gradient multiplied by the learning rate.
        """
        ## TODO: Implement stochastic grad descent for each parameter
        for p in self.params:
          p -= self.lr * p.grad 
        


Below, you'll use your new implementations to optimize for linear regression manually. FakeTorchModule will also be provided to make some of the mimicking process easier. 

**[TODO]:** Complete the model and compare this model's performance to the previous `LinearRegression` model -- they should have a similar loss after training.

In [112]:
import torch
import torch.nn as nn

class FakeTorchModule:
    """
        Needed so that we can do manual linear regression.
    """

    def __init__(self):
        self.device = ""

    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)

    def to(self, device):
        return self

    def parameters(self):
        params = []
        for k,v in self.__dict__.items():
            params += getattr(v, 'parameters', lambda: [])()
        return params

    def train(self):
        for p in self.parameters():
            p.requires_grad = getattr(p, 'required_grad', p.requires_grad)
    
    def eval(self):
        for p in self.parameters():
            p.required_grad = p.requires_grad
            p.requires_grad = False

class ManualRegression(FakeTorchModule):
    """
        Allows us to use our custom Linear layer and SGD optimizer.
        Subclasses FakeTorchModule
    """

    def __init__(self, input_dims, output_dims): 
        super().__init__()
        ## TODO: Incorporate your custom components in the initialization pipeline. 
        self.dense = Linear(input_dims, output_dims, device=None, dtype=None)
        self.activation = None  ## To be specified in subclasses 
        self.loss = None   ## To be specified in subclasses 
        self.set_learning_rate()

    def set_learning_rate(self, learning_rate=0.001):
        ## TODO: Use your new SGD component and make changes as appropriate.
        self.optimizer = SGD(self.dense.parameters(), lr=learning_rate)

    def forward(self, x):
        ## TODO: Implement the forward function as appropriate. Make changes as necessary
        x = self.dense(x)
        return x

class TrainTest2(TrainTest):
    # no_grad = torch.no_grad
    no_grad = Tensor.no_grad

class ManualLinearRegression(ManualRegression, TrainTest2):
    def __init__(self, input_dims, output_dims):
        super().__init__(input_dims, output_dims)
        ## TODO: Implement the subclass as appropriate with your own implementations  
        self.activation = None  ## To be specified in subclasses 
        self.loss = MSELoss()   ## To be specified in subclasses 

## Train the manual linear regression model
model = ManualLinearRegression(10, 1)
model.set_learning_rate(0.3)
model.train_test(
    [[Tensor(X0), Tensor(Y0)]], 
    [[Tensor(X1), Tensor(Y1)]],
    epochs=200
);
## TODO: Compare this model's performance to the first linear regression model -- they should be similar

(1, 1)
[Epoch 1/200] {'loss': Tensor(30504.62032247), 'test_loss': 6573.657911650106}
[Epoch 2/200] {'loss': Tensor(9970.7444139), 'test_loss': 4805.339365524657}
[Epoch 3/200] {'loss': Tensor(6664.57787744), 'test_loss': 4870.014383911763}
[Epoch 4/200] {'loss': Tensor(6114.44194721), 'test_loss': 5009.054403014087}
[Epoch 5/200] {'loss': Tensor(6005.38310702), 'test_loss': 5073.039608849727}
[Epoch 6/200] {'loss': Tensor(5967.08931478), 'test_loss': 5090.535493202507}
[Epoch 7/200] {'loss': Tensor(5940.32166961), 'test_loss': 5086.981643153232}
[Epoch 8/200] {'loss': Tensor(5915.60187665), 'test_loss': 5074.748365900602}
[Epoch 9/200] {'loss': Tensor(5891.41158283), 'test_loss': 5059.1153558128135}
[Epoch 10/200] {'loss': Tensor(5867.5057647), 'test_loss': 5042.239091731735}
[Epoch 11/200] {'loss': Tensor(5843.84310921), 'test_loss': 5024.985255254127}
[Epoch 12/200] {'loss': Tensor(5820.41491264), 'test_loss': 5007.698324301012}
[Epoch 13/200] {'loss': Tensor(5797.21771033), 'test_l

-----

## Wrapping Up

Congratulations, you've finished this assignment! You should now have a better understand of linear regression, loss functions, and optimizers/gradient descent. This assignment provides the foundation for Homework 2, so feel free to come back or read it over again to get a solid understanding.

Be sure to submit your finished notebook (follow the guidelines on the handout).